In [1]:
import pysnooper
import gym
import ray

import os, sys, pickle, time
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# ----------
# Pytorch
import torch
device = torch.device('cuda', index=args.gpu_index) \
    if torch.cuda.is_available() else torch.device('cpu')
# ----------
# flow
from flow.utils.registry import make_create_env
benchmark_name = 'multi_merge'
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.buffered_obs_flow_params

# ----------
# PyTorch-RL
from models.mlp_policy import MultiAgentPolicy
from models.mlp_critic import Value
from core.a2c import a2c_step
from core.common import estimate_advantages
from core.multi_agent import MultiAgent
from utils.remote_vector_env import MultiAgentVecEnv, dict_to_array
from utils.replay_memory import MultiAgentMemory

In [2]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-03_21-02-42_2895/logs.
Waiting for redis server at 127.0.0.1:16167 to respond...
Waiting for redis server at 127.0.0.1:17878 to respond...
Starting the Plasma object store with 3.2789331959999997 GB memory using /dev/shm.

View the web UI at http://localhost:8888/notebooks/ray_ui.ipynb?token=1b6a9a67cbab56060dd78f1264d4e71abc087f3ae86bca49



{'node_ip_address': '168.150.112.131',
 'object_store_addresses': ['/tmp/ray/session_2019-05-03_21-02-42_2895/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-05-03_21-02-42_2895/sockets/raylet'],
 'redis_address': '168.150.112.131:16167',
 'webui_url': 'http://localhost:8888/notebooks/ray_ui.ipynb?token=1b6a9a67cbab56060dd78f1264d4e71abc087f3ae86bca49'}

# Set env

In [3]:
create_env, env_name = make_create_env(params=flow_params, version=0)
sample_env = create_env()
state_dim = sample_env.observation_space.shape[0]
action_dim = sample_env.action_space.shape[0]
env = MultiAgentVecEnv(create_env, num_envs=2, remote_env_batch_wait_ms=0)

 Starting SUMO on port 43405


# Set Policy

In [4]:
# define actor and critic
policy_net = MultiAgentPolicy(state_dim, action_dim, activation='relu')
value_net = Value(state_dim)
optimizer_policy = torch.optim.Adam(policy_net.parameters(), lr=3e-4)
optimizer_value = torch.optim.Adam(value_net.parameters(), lr=3e-4)

# Sampling

In [5]:
state = env.reset()

Launching env 0 in remote actor
Launching env 1 in remote actor


In [6]:
agent = MultiAgent(env, policy_net)

In [7]:
agent.collect_samples(300)

In [8]:
for batch in agent.batch_generator(100):
    print(batch)

Transition(state=array([[ 0.23507847, -0.03121408,  0.01227797, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.31500962,  0.27555642,  0.07981187, ...,  0.63670101,
         0.02417168,  0.09036358],
       [ 0.34319687,  0.15644275,  0.03674331, ...,  0.56882167,
         0.02844178,  0.10131611],
       ...,
       [ 0.25475159,  0.34524387,  0.23197281, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.01858723,  0.98141277,  1.        , ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.04251057,  0.95748943,  1.        , ...,  0.56882167,
         0.02844178,  0.10131611]]), action=array([[-1.0284888 ],
       [ 2.4669693 ],
       [ 0.68278456],
       [ 0.14082204],
       [-0.5532514 ],
       [ 2.0521972 ],
       [-0.17932129],
       [ 0.07350043],
       [-1.446568  ],
       [ 1.111959  ],
       [-0.24582806],
       [ 0.7962627 ],
       [ 0.41744977],
       [ 0.64458895],
       [ 1.1781858 ],
       [ 0.4186976 ],
       [ 1

       0.23403871, 0.38555659, 0.27341337, 0.73100481, 0.4811592 ]))
Transition(state=array([[ 0.36584759, -0.00833358,  0.02765797, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.28280008,  0.00302402,  0.01509452, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.37375197,  0.21283028,  0.16744873, ...,  0.63670101,
         0.02417168,  0.09036358],
       ...,
       [ 0.00829853, -0.00259769,  0.03468396, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.44881022, -0.17979266,  0.03584734, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.05538325,  0.17805989,  0.07956082, ...,  0.63670101,
         0.02417168,  0.09036358]]), action=array([[-1.78743303e-01],
       [-2.85738021e-01],
       [ 1.51255518e-01],
       [ 5.04568256e-02],
       [ 1.71375060e+00],
       [-4.03077930e-01],
       [-3.28032643e-01],
       [-2.51319051e-01],
       [-7.08419502e-01],
       [ 2.09840834e-01],
       [-4.95224148e-02],
       

Transition(state=array([[ 0.02778941,  0.07537961,  0.01527748, ...,  0.63670101,
         0.02417168,  0.09036358],
       [ 0.25621879, -0.02812425,  0.01951373, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.13546911, -0.02990482,  0.01196092, ...,  0.56882167,
         0.02844178,  0.10131611],
       ...,
       [ 0.56146866,  0.20783415,  0.13676675, ...,  0.63670101,
         0.02417168,  0.09036358],
       [ 0.56146866,  0.20783415,  0.13676675, ...,  0.63670101,
         0.02417168,  0.09036358],
       [ 0.0473845 ,  0.08949135,  0.11359808, ...,  0.63670101,
         0.02417168,  0.09036358]]), action=array([[-0.41353688],
       [-0.3657648 ],
       [-1.6667613 ],
       [ 0.57387584],
       [ 0.8905616 ],
       [ 0.03731364],
       [-0.59672976],
       [ 1.0334364 ],
       [-0.07257036],
       [-1.8653073 ],
       [ 1.0034978 ],
       [-0.6172436 ],
       [ 0.93452513],
       [ 1.2008073 ],
       [ 2.4303389 ],
       [-0.18860069],
       [-0

Transition(state=array([[ 0.13136917,  0.03046057,  0.02153869, ...,  0.63670101,
         0.02417168,  0.09036358],
       [ 0.01170137, -0.01170137,  0.00569745, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.22613478,  0.05621454,  0.21389697, ...,  0.63670101,
         0.02417168,  0.09036358],
       ...,
       [ 0.05379084, -0.02266018,  0.00634478, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.01992429,  0.26911825,  0.42238113, ...,  0.56882167,
         0.02844178,  0.10131611],
       [ 0.11843317,  0.04808242,  0.0233792 , ...,  0.63670101,
         0.02417168,  0.09036358]]), action=array([[ 1.4228466 ],
       [ 1.0016253 ],
       [-1.1226642 ],
       [-0.59837425],
       [ 0.168245  ],
       [-0.42036486],
       [-1.2944651 ],
       [ 0.7012771 ],
       [-0.96025854],
       [ 1.1085658 ],
       [-0.861454  ],
       [-1.1134855 ],
       [-0.27138907],
       [ 0.05329081],
       [-0.6249185 ],
       [ 0.71434325],
       [-0

       0.23880742, 0.68921682, 0.3383016 , 0.2378332 , 0.46940826]))
Transition(state=array([[ 5.84421158e-01,  1.92954555e-01,  1.01614863e-01, ...,
         6.36701009e-01,  2.41716757e-02,  9.03635822e-02],
       [ 1.93804726e-01,  1.49112582e-01,  1.37223214e-01, ...,
         6.36701009e-01,  2.41716757e-02,  9.03635822e-02],
       [ 1.54963788e-03, -1.54963788e-03,  5.16117085e-03, ...,
         5.68821673e-01,  2.84417768e-02,  1.01316114e-01],
       ...,
       [ 1.56981960e-01,  1.51284471e-01,  1.80581227e-01, ...,
         6.36701009e-01,  2.41716757e-02,  9.03635822e-02],
       [ 2.10307702e-01,  4.34463061e-02,  2.16600478e-01, ...,
         6.36701009e-01,  2.41716757e-02,  9.03635822e-02],
       [ 2.63983083e-07,  7.49999762e-01,  3.48141223e-01, ...,
         5.68821673e-01,  2.84417768e-02,  1.01316114e-01]]), action=array([[-1.7725102 ],
       [-0.56894535],
       [-0.55758023],
       [-0.34022704],
       [-0.31292588],
       [ 0.56770366],
       [-0.698770

       0.60551504, 0.79976807, 0.23004733, 0.59441472, 0.44837501]))
Transition(state=array([[0.01181958, 0.04339728, 0.01043209, ..., 0.56882167, 0.02844178,
        0.10131611],
       [0.05364354, 0.27492854, 0.35488787, ..., 0.56882167, 0.02844178,
        0.10131611],
       [0.31685442, 0.25134271, 0.21095002, ..., 0.63670101, 0.02417168,
        0.09036358],
       ...,
       [0.01831811, 0.98168188, 1.        , ..., 0.56882167, 0.02844178,
        0.10131611],
       [0.04775258, 0.0085567 , 0.01169192, ..., 0.56882167, 0.02844178,
        0.10131611],
       [0.243747  , 0.0961334 , 0.20018584, ..., 0.63670101, 0.02417168,
        0.09036358]]), action=array([[-1.7084104 ],
       [-0.41081768],
       [-0.04693461],
       [ 1.5320135 ],
       [-1.5143188 ],
       [-1.1790277 ],
       [ 3.064798  ],
       [ 0.10213937],
       [-1.0870044 ],
       [ 0.20353502],
       [ 0.1793683 ],
       [-0.31622288],
       [-0.7924609 ],
       [ 0.4924247 ],
       [ 0.18520144],

In [16]:
batch.action

array([[ 1.0089955 ],
       [ 1.0857189 ],
       [ 1.0546966 ],
       [-0.50386685],
       [-0.1240186 ],
       [-0.8009682 ],
       [-0.55243236],
       [-0.5326888 ],
       [-1.6586194 ],
       [-0.6289528 ],
       [-0.09865022],
       [-0.49876428],
       [ 2.152362  ],
       [ 1.793769  ],
       [-0.15809107],
       [-0.14993782],
       [ 2.1568115 ],
       [ 0.6205706 ],
       [ 0.14411354],
       [-0.3873442 ],
       [-0.95379007],
       [-0.9049608 ],
       [ 0.46557128],
       [-1.479171  ],
       [-0.0650925 ],
       [-0.16842227],
       [ 0.22430387],
       [ 1.1946125 ],
       [-1.7946342 ],
       [ 1.6216971 ],
       [ 1.9812596 ],
       [-1.264636  ],
       [-0.2769514 ],
       [-0.37149706],
       [ 2.1606398 ],
       [-0.29537165],
       [ 0.7461815 ],
       [-0.01317039],
       [-0.64112854],
       [-1.4918777 ],
       [ 0.9487128 ],
       [-0.41120243],
       [ 0.902652  ],
       [ 0.07543186],
       [-0.50682706],
       [-0

# Debug

In [9]:
# <Seeding should be here>

In [55]:
agent.memory = MultiAgentMemory()
state = agent.env.reset()
for i in range(1000):
    with torch.no_grad():
        action = agent.policy_net.select_action(state)
    next_state, rew, done, info = agent.env.step(action)
    # if __all__ True, reset
    need_reset = list(map(lambda value:1 if value["__all__"] else 0,\
                                  list(done.values())))
    action, state, listed_next_state, rew, done, info, id_list = \
            dict_to_array(action, state, next_state, rew, done, info)
    mask = list(map(lambda d: 0 if d else 1, done))    
    agent.memory.push(state, action, mask, listed_next_state, rew)
    state = next_state
    # reset if need_reset is 1
    reseted_state = env.reset(need_reset)
    for key, value in reseted_state.items():
        state[key] = value

env: 0 is reseted at 699
env: 1 is reseted at 699
